In [1]:
import torch
import numpy as np
import matchzoo as mz
print('matchzoo version', mz.__version__)

matchzoo version 1.1.1


In [2]:
ranking_task = mz.tasks.Ranking(losses=mz.losses.RankHingeLoss(num_neg=4))
ranking_task.metrics = [
    mz.metrics.NormalizedDiscountedCumulativeGain(k=3),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=5),
    mz.metrics.MeanAveragePrecision()
]
print("`ranking_task` initialized with metrics", ranking_task.metrics)

`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]


In [3]:
print('data loading ...')
train_pack_raw = mz.datasets.wiki_qa.load_data('train', task=ranking_task)
dev_pack_raw = mz.datasets.wiki_qa.load_data('dev', task=ranking_task, filtered=True)
test_pack_raw = mz.datasets.wiki_qa.load_data('test', task=ranking_task, filtered=True)
print('data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`')

data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`


In [4]:
preprocessor = mz.models.DIIN.get_default_preprocessor(
    truncated_length_left=30,
    truncated_length_right=30,
    ngram_size=1
)

In [5]:
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
dev_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 8619.03it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:04<00:00, 4657.98it/s]
Building Vocabulary from a datapack.: 100%|██████████| 2919765/2919765 [00:00<00:00, 3962844.40it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 8879.66it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:04<00:00, 4523.80it/s]
Processing length_right with len: 100%|██████████| 18841/18841 [00:00<00:00, 739400.26it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 122/122 [00:00<00:00, 8604.57it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 1115/1115 [00:00<

In [6]:
preprocessor.context

{'ngram_process_unit': <matchzoo.preprocessors.units.ngram_letter.NgramLetter at 0x7eff65d292e8>,
 'filter_unit': <matchzoo.preprocessors.units.frequency_filter.FrequencyFilter at 0x7eff66145828>,
 'vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x7eff65a961d0>,
 'vocab_size': 30058,
 'embedding_input_dim': 30058,
 'ngram_vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x7eff5f003358>,
 'ngram_vocab_size': 179}

In [7]:
glove_embedding = mz.datasets.embeddings.load_glove_embedding(dimension=100)
term_index = preprocessor.context['vocab_unit'].state['term_index']
embedding_matrix = glove_embedding.build_matrix(term_index)
l2_norm = np.sqrt((embedding_matrix * embedding_matrix).sum(axis=1))
embedding_matrix = embedding_matrix / l2_norm[:, np.newaxis]

In [8]:
ngram_callback = mz.dataloader.callbacks.Ngram(preprocessor, mode='index')
trainset = mz.dataloader.Dataset(
    data_pack=train_pack_processed,
    mode='pair',
    num_dup=2,
    num_neg=4,
    batch_size=16,
    resample=True,
    sort=False,
    shuffle=True,
    callbacks=[ngram_callback]
)
devset = mz.dataloader.Dataset(
    data_pack=dev_pack_processed,
    mode='point',
    batch_size=16,
    resample=False,
    sort=False,
    shuffle=True,
    callbacks=[ngram_callback]
)

In [9]:
padding_callback = mz.models.DIIN.get_default_padding_callback(
    fixed_length_left=30,
    fixed_length_right=30,
    fixed_ngram_length=15,
)
trainloader = mz.dataloader.DataLoader(
    dataset=trainset,
    stage='train',
    callback=padding_callback
)
devloader = mz.dataloader.DataLoader(
    dataset=devset,
    stage='dev',
    callback=padding_callback
)

In [10]:
model = mz.models.DIIN()

model.params['task'] = ranking_task
model.params['embedding'] = embedding_matrix
model.params['mask_value'] = 0
model.params['char_embedding_input_dim'] = preprocessor.context['ngram_vocab_size']
model.params['char_embedding_output_dim'] = 8
model.params['char_conv_filters'] = 100
model.params['char_conv_kernel_size'] = 5
model.params['first_scale_down_ratio'] = 0.3
model.params['nb_dense_blocks'] = 3
model.params['layers_per_dense_block'] = 8
model.params['growth_rate'] = 20
model.params['transition_scale_down_ratio'] = 0.5
model.params['conv_kernel_size'] = (3, 3)
model.params['pool_kernel_size'] = (2, 2)
model.params['dropout_rate'] = 0.2

model.build()

print(model)
print('Trainable params: ', sum(p.numel() for p in model.parameters() if p.requires_grad))

DIIN(
  (embedding): Embedding(30058, 100, padding_idx=0)
  (char_embedding): CharacterEmbedding(
    (char_embedding): Embedding(179, 8)
    (conv): Conv1d(8, 100, kernel_size=(5,), stride=(1,))
  )
  (exact_maching): Matching()
  (left_encoder): SemanticComposite(
    (att_linear): Linear(in_features=603, out_features=1, bias=False)
    (z_gate): Linear(in_features=402, out_features=201, bias=True)
    (r_gate): Linear(in_features=402, out_features=201, bias=True)
    (f_gate): Linear(in_features=402, out_features=201, bias=True)
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (right_encoder): SemanticComposite(
    (att_linear): Linear(in_features=603, out_features=1, bias=False)
    (z_gate): Linear(in_features=402, out_features=201, bias=True)
    (r_gate): Linear(in_features=402, out_features=201, bias=True)
    (f_gate): Linear(in_features=402, out_features=201, bias=True)
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (matching): Matching()
  (conv): Conv2d(201, 60, kern

In [11]:
optimizer = torch.optim.Adam(model.parameters())
trainer = mz.trainers.Trainer(
    model=model,
    optimizer=optimizer,
    trainloader=trainloader,
    validloader=devloader,
    validate_interval=None,
    epochs=10
)

In [12]:
trainer.run()

[Iter-128 Loss-0.648]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6077 - normalized_discounted_cumulative_gain@5(0.0): 0.6684 - mean_average_precision(0.0): 0.6249



[Iter-256 Loss-0.270]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6093 - normalized_discounted_cumulative_gain@5(0.0): 0.6486 - mean_average_precision(0.0): 0.6037



[Iter-384 Loss-0.092]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6225 - normalized_discounted_cumulative_gain@5(0.0): 0.6744 - mean_average_precision(0.0): 0.6305



[Iter-512 Loss-0.052]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6191 - normalized_discounted_cumulative_gain@5(0.0): 0.6693 - mean_average_precision(0.0): 0.6248



[Iter-640 Loss-0.029]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6043 - normalized_discounted_cumulative_gain@5(0.0): 0.6665 - mean_average_precision(0.0): 0.6234



[Iter-768 Loss-0.018]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6072 - normalized_discounted_cumulative_gain@5(0.0): 0.6685 - mean_average_precision(0.0): 0.6149



[Iter-896 Loss-0.016]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6044 - normalized_discounted_cumulative_gain@5(0.0): 0.6586 - mean_average_precision(0.0): 0.6217



[Iter-1024 Loss-0.009]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5738 - normalized_discounted_cumulative_gain@5(0.0): 0.6371 - mean_average_precision(0.0): 0.5924



[Iter-1152 Loss-0.009]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6089 - normalized_discounted_cumulative_gain@5(0.0): 0.674 - mean_average_precision(0.0): 0.6288



[Iter-1280 Loss-0.014]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6299 - normalized_discounted_cumulative_gain@5(0.0): 0.6822 - mean_average_precision(0.0): 0.6427

Cost time: 1179.0480372905731s
